In [ ]:
# This is a project to use the Py-Coin-Gecko Python Client/Wrapper to access the Coin Gecko API, updated every minute by Coin-Gecko.
#In this function we are getting data on bitcoin, in U.S. Dollars, for the past 30 days and creating a candlestick graph displaying 
# the  minimum, maximum, first, and last price of each day across 24 observations per day (one per hour), grouped by date.

In [41]:
!pip install plotly
!pip install pycoingecko
!pip install mplfinance

In [42]:
import plotly.graph_objects as go
import plotly.offline as plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [43]:
cg = CoinGeckoAPI()

In [44]:
# id is the name of the coin we want,
# vs_currency is the currency we want the price in,
# and days is how many days back from today we want
bitcoin_data = cg.get_coin_market_chart_by_id(id ='bitcoin', vs_currency = 'usd', days = 30) 

In [45]:
type(bitcoin_data)

dict

In [46]:
# The response we get is a JSON document 
# consisting of prices, market caps, and total volumes along with timestamps for each observation
# but we are focussed only on the prices 
bitcoin_price_data = bitcoin_data["prices"]

In [32]:
bitcoin_price_data[:5]

[[1625108629526, 34382.3141936889],
 [1625112170502, 34288.151547988935],
 [1625115738311, 34443.22794538035],
 [1625122933851, 33397.75273444783],
 [1625126502785, 33531.93427401931]]

In [47]:
# Turn into pandas dataframe
data = pd.DataFrame(bitcoin_price_data, columns = ["Timestamp", "Price"])

In [48]:
data.head(5)

,Timestamp,Price
0,1625108629526,34382.314194
1,1625112170502,34288.151548
2,1625115738311,34443.227945
3,1625122933851,33397.752734
4,1625126502785,33531.934274


In [49]:
# convert timestamp to date and store it as a column called Date
data["Date"] = data["Timestamp"].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [50]:
data[:5]

,Timestamp,Price,Date
0,1625108629526,34382.314194,2021-06-30
1,1625112170502,34288.151548,2021-07-01
2,1625115738311,34443.227945,2021-07-01
3,1625122933851,33397.752734,2021-07-01
4,1625126502785,33531.934274,2021-07-01


In [51]:
data.drop("Timestamp",1)

,Price,Date
0,34382.314194,2021-06-30
1,34288.151548,2021-07-01
2,34443.227945,2021-07-01
3,33397.752734,2021-07-01
4,33531.934274,2021-07-01
...,...,...
716,41314.594012,2021-07-30
717,41936.261479,2021-07-30
718,41937.334184,2021-07-30
719,41964.840673,2021-07-30


In [52]:
candlestick_data = data.groupby(data.Date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [53]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['Date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()